# Detect scene changes in videos

Automatically find scene cuts, transitions, and fades in video files.

## Problem

You have video files and need to identify scene boundaries for:

| Use case | Goal |
|----------|------|
| Video editing | Find cut points automatically |
| Content analysis | Segment videos into scenes |
| Thumbnail generation | Extract one frame per scene |
| Clip extraction | Split videos at scene boundaries |
| Ad detection | Find commercial breaks |

## Solution

**What's in this recipe:**

- Detect hard cuts with `scene_detect_content()`
- Find fade transitions with `scene_detect_threshold()`
- Use adaptive detection with `scene_detect_adaptive()`

Three built-in detection methods handle different transition types using PySceneDetect.

### Setup

In [ ]:
%pip install -qU pixeltable scenedetect opencv-python

In [2]:
import pixeltable as pxt

In [3]:
# Create a fresh directory
pxt.drop_dir('scene_demo', force=True)
pxt.create_dir('scene_demo')

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/pjlb/.pixeltable/pgdata
Created directory 'scene_demo'.


### Load sample videos

In [4]:
# Create a video table
videos = pxt.create_table(
    'scene_demo.videos',
    {'video': pxt.Video, 'title': pxt.String}
)

# Insert sample videos from S3
videos.insert([
    {
        'video': 's3://multimedia-commons/data/videos/mp4/ffe/ffb/ffeffbef41bbc269810b2a1a888de.mp4',
        'title': 'Sample video 1'
    }
])

Created table 'videos'.
Inserting rows into `videos`: 1 rows [00:00, 200.53 rows/s]
Inserted 1 row with 0 errors.


1 row inserted, 3 values computed.

### Detect scenes with content-based detection

In [5]:
# Detect scenes using content-based detection (best for hard cuts)
videos.add_computed_column(
    scenes_content=videos.video.scene_detect_content(
        threshold=27.0,      # Lower = more sensitive
        min_scene_len=15     # Minimum frames between cuts
    )
)

# View detected scenes
videos.select(videos.title, videos.scenes_content).collect()

Added 1 column value with 0 errors.


title,scenes_content
Sample video 1,"[{""duration"": 25.692, ""start_pts"": 0, ""start_time"": 0.}]"


### Detect fade transitions

In [6]:
# Detect fade-to-black/white transitions
videos.add_computed_column(
    scenes_fade=videos.video.scene_detect_threshold(
        threshold=12.0,      # Brightness threshold for fades
        min_scene_len=15
    )
)

# View fade-detected scenes
videos.select(videos.title, videos.scenes_fade).collect()

Added 1 column value with 0 errors.


title,scenes_fade
Sample video 1,"[{""duration"": 25.692, ""start_pts"": 0, ""start_time"": 0.}]"


### Adaptive detection for complex videos

In [7]:
# Adaptive detection adjusts to video content dynamically
videos.add_computed_column(
    scenes_adaptive=videos.video.scene_detect_adaptive(
        adaptive_threshold=3.0,  # Lower = more scenes detected
        min_scene_len=15,
        fps=2.0                  # Analyze at 2 FPS for speed
    )
)

# View adaptively-detected scenes
videos.select(videos.title, videos.scenes_adaptive).collect()

Added 1 column value with 0 errors.


title,scenes_adaptive
Sample video 1,"[{""duration"": 25.526, ""start_pts"": 0, ""start_time"": 0.}]"


## Explanation

**Detection methods:**

| Method | Best for | Key parameter |
|--------|----------|---------------|
| `scene_detect_content()` | Hard cuts | `threshold` (27 default) |
| `scene_detect_threshold()` | Fades | `threshold` (12 default) |
| `scene_detect_adaptive()` | Mixed content | `adaptive_threshold` (3 default) |

**Output format:**

Each method returns a list of scene dictionaries:

```python
{
    'start_time': 5.2,    # Scene start in seconds
    'start_pts': 156,     # Presentation timestamp
    'duration': 3.8       # Scene duration in seconds
}
```

**Tuning tips:**

| Goal | Adjustment |
|------|------------|
| More scenes | Lower threshold values |
| Fewer scenes | Higher threshold values |
| Faster processing | Set `fps=1.0` or `fps=2.0` |
| Ignore quick cuts | Increase `min_scene_len` |

## See also

- [Extract frames from videos](https://docs.pixeltable.com/howto/cookbooks/video/video-extract-frames) - Get frames at scene boundaries
- [Generate thumbnails](https://docs.pixeltable.com/howto/cookbooks/video/video-generate-thumbnails) - Create preview images